# RedCap Status Dashboard - One Line Per Form
Creating a dashboard of one form per line

### Parametrization

In [1]:
site: str
arm: str
form: str

### Import Libraries
Set up libraries to display each form and navigate paths to find appropriate files

In [2]:
from IPython.display import display, Markdown, Latex
import pandas as pd
from pathlib import Path
import sys
sys.path.append('/sibis-software/ncanda-data-integration/scripts/qc/')
import filter_inventory

### Display Form
Function applies each of the filters in filter_inventories, and displays the function with the appropriate style

In [3]:
def show_form(form_data, site, arm, form):
    # List of filter functions to apply -> check scripts/filter_inventories.py
    FILTER_LIST = [
            'empty_marked_present',
            'content_marked_missing',
            'less_content_than_max',
            'empty_unmarked',
            'content_unmarked',
            'content_not_complete',
            'missing_not_complete',
            'excluded_with_content',
        ]
    
    # Apply filters to dataframe
    for filter in FILTER_LIST:
        form_data[filter] = eval('filter_inventory.' + filter + '(form_data)')

    # Go from Booleans to Icons
    form_data = form_data.applymap(lambda x: '✅' if x is False else x).applymap(lambda x: '❌' if x is True else x)
    
    # Transform -  transform: rotate(-90deg);
    table_styles = [{
        'selector': 'td',
        'props': [('background-color', '#808080'), ('color', '#FFFFFF')]
    }, 
    {
        'selector': 'th',
        'props': [('background-color', '#000000'), ('color', '#FF0000')]
    },
    {
        'selector': 'th.col_heading',
        'props': [('transform', 'rotate(-90deg)')]
    }]
    # Display form data with markdown
    display(Markdown('## ' + site.capitalize() + ' Dashboard'))
    display(Markdown(form + ' form dashboard for arm ' + arm))
    display(form_data.style.set_table_styles(table_styles).set_table_styles(
    [dict(selector="th",props=[('max-width', '50px')]),
        dict(selector="th.col_heading",
                 props=[("writing-mode", "vertical-lr")])]))

### Arm + Form Parser
Parses through each arm and form for each site, and then displays the results to the screen

In [4]:
def iter_sites(sites, arms, forms):
    # Goes through each site -> arm -> form
    for site in sites:
        p = Path('/fs/ncanda-share/log/make_all_inventories/inventory_by_site/' + str(site))
        for arm_year in p.iterdir():
            if (arm_year.is_dir() and arm_year.stem in arms):
                for form in arm_year.iterdir():
                    if (form.is_dir() == False and form.stem in forms):
                        # Reads as Pandas dataframe and calls function to
                        # show form
                        df = pd.read_csv(form)
                        if ('missing'  in df.columns):
                            show_form(df, site, arm_year.stem, form.stem)

## Main Function
Passes in the specific sites, arms, and forms to look at, and then runs everything accordingly

In [5]:
def main():
    sites = [site]
    arms = [arm]
    forms = [form]
    iter_sites(sites, arms, forms)

### Run Main

In [6]:
main()

## Duke Dashboard

mri_stroop form dashboard for arm 1y_visit_arm_1

,study_id,redcap_event_name,dag,non_nan_count,exclude,missing,complete,form_name,status,empty_marked_present,content_marked_missing,less_content_than_max,empty_unmarked,content_unmarked,content_not_complete,missing_not_complete,excluded_with_content
0,C-70001-F-3,1y_visit_arm_1,duke,0.000000,1.000000,nan,0.000000,mri_stroop,EMPTY,✅,✅,✅,✅,✅,✅,✅,✅
1,C-70002-F-4,1y_visit_arm_1,duke,0.000000,1.000000,nan,0.000000,mri_stroop,EMPTY,✅,✅,✅,✅,✅,✅,✅,✅
2,C-70003-F-6,1y_visit_arm_1,duke,0.000000,0.000000,1.000000,2.000000,mri_stroop,MISSING,✅,✅,✅,✅,✅,✅,✅,✅
3,C-70003-F-6,1y_visit_arm_1,duke,0.000000,0.000000,1.000000,2.000000,mri_stroop,MISSING,✅,✅,✅,✅,✅,✅,✅,✅
4,C-70004-F-5,1y_visit_arm_1,duke,0.000000,1.000000,nan,0.000000,mri_stroop,EMPTY,✅,✅,✅,✅,✅,✅,✅,✅
5,C-70006-M-3,1y_visit_arm_1,duke,0.000000,0.000000,1.000000,2.000000,mri_stroop,MISSING,✅,✅,✅,✅,✅,✅,✅,✅
6,C-70007-M-7,1y_visit_arm_1,duke,0.000000,1.000000,nan,0.000000,mri_stroop,EMPTY,✅,✅,✅,✅,✅,✅,✅,✅
7,C-70007-M-7,1y_visit_arm_1,duke,0.000000,1.000000,nan,0.000000,mri_stroop,EMPTY,✅,✅,✅,✅,✅,✅,✅,✅
8,C-70008-F-6,1y_visit_arm_1,duke,0.000000,0.000000,1.000000,2.000000,mri_stroop,MISSING,✅,✅,✅,✅,✅,✅,✅,✅
9,C-70010-F-1,1y_visit_arm_1,duke,0.000000,0.000000,1.000000,2.000000,mri_stroop,MISSING,✅,✅,✅,✅,✅,✅,✅,✅
